## Functions for plotting metrics for comparison (text in Polish)

> Applies to local experiments not described in the code or documentation.

# Helpers

### Prep

In [ ]:
import sys
import functools
from pathlib import Path
from typing import Optional, Sequence, Union, Any, Callable, Mapping, Tuple

import numpy as np
import pandas as pd
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

DEFAULT_FONTSIZE_LARGE = 18
DEFAULT_FONTSIZE_MEDIUM = 16
DEFAULT_FONTSIZE_SMALL = 13

if 'my_init_done' not in globals():
    my_init_done = True

    try:
        here = Path(__file__).resolve().parent
    except NameError:
        here = Path.cwd()
    sys.path.insert(0, str(here.parent))

def load_styles():
    sns.set_style("ticks", {
        "axes.grid": True,
        "grid.linestyle": "--",
        "grid.alpha": 0.4
    })
    sns.set_context("paper", font_scale=1.0)

    plt.rc('font', family='serif', serif=['Times New Roman', 'Computer Modern'])
    
    plt.rcParams.update({
        "figure.figsize": (8, 4),
        "figure.dpi": 300,
        "savefig.dpi": 300,

        "axes.linewidth": 1.0,
        "axes.labelsize": DEFAULT_FONTSIZE_LARGE,
        "axes.titlepad": 6,

        "xtick.direction": "in",
        "ytick.direction": "in",
        "xtick.major.size": 5,
        "ytick.major.size": 5,
        "xtick.minor.size": 3,
        "ytick.minor.size": 3,
        "xtick.labelsize": DEFAULT_FONTSIZE_MEDIUM,
        "ytick.labelsize": DEFAULT_FONTSIZE_MEDIUM,
        "xtick.bottom": True,
        "xtick.top": False,
        "ytick.left": True,
        "ytick.right": False,

        "lines.linewidth": 1.0,
        "lines.markersize": 6,

        "legend.frameon": False,
        "legend.fontsize": DEFAULT_FONTSIZE_SMALL,
        "legend.loc": "lower right",

        "grid.linewidth": 0.5,
    })
    
load_styles()   # Always load styles for plots

### Tables

In [ ]:
FilterValue = Union[
    Any,
    Sequence[Any],
    Callable[[pd.Series], pd.Series]
]

def _make_mask(df: pd.DataFrame, conds: Mapping[str, FilterValue]) -> pd.Series:
    mask = pd.Series(True, index=df.index)
    for col, cond in conds.items():
        if callable(cond):
            m = cond(df[col])
        elif isinstance(cond, (list, tuple, set, pd.Index)):
            m = df[col].isin(cond)
        else:
            m = df[col] == cond
        mask &= m
    return mask

def filter_df(df: pd.DataFrame, filters: Mapping[str,FilterValue]):
    df2 = df
    masks = [_make_mask(df2, fgroup) for fgroup in filters]
    full_mask = functools.reduce(lambda a,b: a|b, masks)
    df2 = df2.loc[full_mask]
    return df2

def read_csv_numeric_pandas(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, sep=',', quotechar='"', dtype=str)
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            continue
    return df

def print_pretty_table(df: pd.DataFrame, columns: Optional[Sequence[str]] = None,
                       sort_by: Optional[Union[str, Sequence[str]]] = None,
                       headers: str = "keys", tablefmt: str = "psql",
                       showindex: bool = False) -> None:
    df2 = df

    if columns is not None:
        df2 = df2.loc[:, columns]
        
    if sort_by is not None:
        df2 = df2.sort_values(by=sort_by, ascending=True)
    
    s = tabulate(df2, headers=headers, tablefmt=tablefmt, showindex=showindex,
                 floatfmt=".4g")
    print(s)

### Line plots (x2)

In [ ]:
NumberFormatter = Callable[[float], str]

def plot_2lines(df: pd.DataFrame, x: str, y1: str, y2: str, *,
                y1_label: Optional[str] = None, y2_label: Optional[str] = None,
                xlabel: Optional[str] = None, ylabel: Optional[str] = None,
                y_multiplier: float = 1.0, y_suffix: str = "",
                ylim: Optional[Tuple[float, float]] = None,
                legend_loc: str = "best", fmt: Optional[NumberFormatter] = None,
                annotate: bool = False, line_palette: Union[str, Sequence] = "tab10",
                markers: Optional[Sequence[str]] = None, save_path: Optional[str] = None,
                show: bool = True) -> Tuple[plt.Figure, plt.Axes]:
    df2 = df[[x, y1, y2]].dropna().sort_values(by=x)
    xs = df2[x].values
    y1s = df2[y1].values
    y2s = df2[y2].values

    def _default_fmt(v: float) -> str:
        s = f"{v:.2f}".replace(".", ",")
        return f"{s}{y_suffix}"
    fmt = fmt or _default_fmt
    def _wrapped_fmt(v: float) -> str:
        return fmt(v * y_multiplier)

    if isinstance(line_palette, str):
        colors = sns.color_palette(line_palette, 2)
    else:
        colors = list(line_palette)
    default_markers = ["o", "s"]
    if markers is None:
        markers = default_markers
    elif len(markers) < 2:
        raise ValueError("You need at least 2 markers")

    fig, ax = plt.subplots()
    ax.plot(xs, y1s, color=colors[0], marker=markers[0], linewidth=2, markersize=4, label=y1_label or y1)
    ax.plot(xs, y2s, color=colors[1], marker=markers[1], linewidth=2, markersize=4, label=y2_label or y2)

    if annotate:
        for xi, yi in zip(xs, y1s):
            ax.annotate(_wrapped_fmt(yi), xy=(xi, yi), xytext=(0,5),
                        textcoords="offset points", ha="center", va="bottom")
        for xi, yi in zip(xs, y2s):
            ax.annotate(_wrapped_fmt(yi), xy=(xi, yi), xytext=(0,5),
                        textcoords="offset points", ha="center", va="bottom")

    if xlabel: ax.set_xlabel(xlabel)
    if ylabel: ax.set_ylabel(ylabel)
    
    xt = sorted(df2[x].unique())
    if len(xt) > 20:
        start, end = xt[0], xt[-1]
        step = 10
        ticks = [start]
        first = ((start // step) + 1) * step
        ticks += list(range(first, end + 1, step))
        ax.set_xticks(ticks)
    else:
        ax.set_xticks(xt)

    if ylim:
        ax.set_ylim(*ylim)
    else:
        y0, y1_ = min(y1s.min(), y2s.min()), max(y1s.max(), y2s.max())
        pad = (y1_ - y0) * 0.1 if y1_ != y0 else 1
        ax.set_ylim(y0 - pad, y1_ + pad)

    ax.yaxis.set_major_formatter(FuncFormatter(lambda v, pos: _wrapped_fmt(v)))
    ax.legend(frameon=False, loc=legend_loc)

    sns.despine(top=True, right=True)
    fig.tight_layout()
    if save_path:
        fig.savefig(save_path)
    if show:
        plt.show()
    return fig, ax

### Bar Plots (x1)

In [ ]:
NumberFormatter = Callable[[float], str]

def plot_bar(df: pd.DataFrame, x: str, y: str, *,
             kind: str = "bar", hue: Optional[str] = None,                       
             palette: Union[str, Sequence] = "tab10", cmap: str = "viridis",                        
             xlabel: Optional[str] = None, ylabel: Optional[str] = None,
             hue_label: Optional[str] = None, y_multiplier: float = 1.0,
             y_suffix: str = "", ylim: Optional[Tuple[float, float]] = None,
             pallim: Optional[Tuple[float, float]] = None,
             fmt: Optional[NumberFormatter] = None, annotate: bool = True,
             width: float = 0.8, save_path: Optional[str] = None,
             show: bool = True) -> Tuple[plt.Figure, plt.Axes]:
    cols = [x, y] + ([hue] if hue is not None else [])
    df2 = df[cols].dropna().copy()
    df2 = df2.sort_values(by=x)
    xs = df2[x].values
    ys = df2[y].values

    def _default_fmt(v: float) -> str:
        s = f"{v:.2f}".replace(".", ",")
        return f"{s}{y_suffix}"
    fmt = fmt or _default_fmt
    def _wrapped_fmt(raw_v: float) -> str:
        return fmt(raw_v * y_multiplier)

    bar_colors = None
    color_lut = None

    if hue is not None and hue in df2.columns:
        vals = df2[hue]
        if pd.api.types.is_numeric_dtype(vals):
            print(f"[{vals.min()}, {vals.max()}]")
            if pallim is None:
                pallim = [vals.min(), vals.max()]
            norm = Normalize(vmin=pallim[0], vmax=pallim[1])
            cmap_obj = plt.get_cmap(cmap)
            bar_colors = cmap_obj(norm(vals.values))
        else:
            cats = vals.unique().tolist()
            pal = sns.color_palette(palette, len(cats))
            color_lut = dict(zip(cats, pal))
            bar_colors = vals.map(color_lut).tolist()
    else:
        bar_colors = sns.color_palette(palette, len(xs))

    legend_title = hue_label or hue
    fig, ax = plt.subplots()

    if kind == "bar":
        bars = ax.bar(
            xs, ys,
            color=bar_colors,
            edgecolor="black",
            width=width,
            zorder=1
        )

        for sp in ax.spines.values():
            sp.set_zorder(2)
            sp.set_clip_on(False)

        xmin, xmax = xs.min(), xs.max()
        pad = width * 1.0
        ax.set_xlim(xmin - pad, xmax + pad)

        if color_lut is not None:
            import matplotlib.patches as mpatches
            handles = [
                mpatches.Patch(color=color_lut[c], label=str(c))
                for c in color_lut
            ]
            ax.legend(handles=handles,
                      title=legend_title, 
                      loc="best")

        if pd.api.types.is_numeric_dtype(df2[hue]) if hue is not None else False:
            sm = ScalarMappable(norm=norm, cmap=cmap_obj)
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=ax,
                                fraction=0.046, pad=0.04)
            cbar.set_label(legend_title) 
            cbar.ax.tick_params(labelsize=DEFAULT_FONTSIZE_SMALL)
    else:
        line_color = bar_colors[0] if isinstance(bar_colors, (list, tuple)) else bar_colors
        ax.plot(xs, ys, color=line_color, marker="o", linewidth=2)
        ax.margins(x=0.05)

    if annotate and kind == "bar":
        for bar in bars:
            h = bar.get_height()
            ax.annotate(
                _wrapped_fmt(h),
                xy=(bar.get_x() + bar.get_width() / 2, h),
                xytext=(0, 5),
                textcoords="offset points",
                ha="center", va="bottom",
                fontsize=DEFAULT_FONTSIZE_SMALL
            )
    elif annotate:
        for xi, yi in zip(xs, ys):
            ax.annotate(
                _wrapped_fmt(yi),
                xy=(xi, yi),
                xytext=(0, 5),
                textcoords="offset points",
                ha="center", va="bottom",
                fontsize=DEFAULT_FONTSIZE_SMALL
            )

    if xlabel:
        ax.set_xlabel(xlabel, labelpad=10)
    if ylabel:
        ax.set_ylabel(ylabel, labelpad=10)

    ax.set_xticks(xs)
    if ylim is not None:
        ax.set_ylim(*ylim)
    else:
        ymin0, ymax0 = ys.min(), ys.max()
        ypad = (ymax0 - ymin0) * 0.1 if ymax0 > ymin0 else 1
        ax.set_ylim(ymin0 - ypad, ymax0 + ypad)

    ax.yaxis.set_major_formatter(FuncFormatter(lambda val, pos: _wrapped_fmt(val)))

    sns.despine(top=True, right=True)
    fig.tight_layout()

    if save_path:
        fig.savefig(save_path)
    if show:
        plt.show()

### Line Plots (x3)

In [ ]:
NumberFormatter = Callable[[float], str]

def plot_3lines_hue(df: pd.DataFrame, x: str, y: str, z: str, hue: str, *,
                    z_label: Optional[str] = None, hue_label: Optional[str] = None,           
                    xlabel: Optional[str] = None, ylabel: Optional[str] = None,
                    y_multiplier: float = 1.0, y_suffix: str = "",
                    ylim: Optional[Tuple[float, float]] = None,
                    pallim: Optional[Tuple[float, float]] = None,
                    fmt: Optional[NumberFormatter] = None,
                    annotate: bool = False, palette: Union[str, Sequence] = "tab10",     
                    cmap: str = "viridis", markers: Optional[Sequence[str]] = None,
                    line_palette: Union[str, Sequence] = "tab10",
                    save_path: Optional[str] = None,
                    show: bool = True) -> Tuple[plt.Figure, plt.Axes]:
    if z not in df.columns or hue not in df.columns:
        raise KeyError(f"Missing column z= '{z} 'or hue=' {hue}'")
    df2 = df[[x, y, z, hue]].dropna().sort_values(by=x).copy()

    def _default_fmt(v: float) -> str:
        s = f"{v:.2f}".replace(".", ",")
        return f"{s}{y_suffix}"
    fmt = fmt or _default_fmt
    def _wrapped_fmt(raw_v: float) -> str:
        return fmt(raw_v * y_multiplier)

    is_num = pd.api.types.is_numeric_dtype(df2[hue])
    if is_num:
        print(f"[{df2[hue].min()}, {df2[hue].max()}]")
        if pallim is None:
            pallim = [df2[hue].min(), df2[hue].max()]
        norm     = Normalize(vmin=pallim[0], vmax=pallim[1])
        cmap_obj = plt.get_cmap(cmap)
    else:
        cats = df2[hue].unique().tolist()
        pal  = sns.color_palette(palette, len(cats))
        lut  = dict(zip(cats, pal))

    groups = df2[z].unique().tolist()
    ng     = len(groups)

    default_markers = ["o","s","^","D","v","<",">","p","X","*"]
    if markers is None:
        markers = (default_markers * ((ng//len(default_markers))+1))[:ng]
    elif len(markers) < ng:
        raise ValueError(f"You need at least {ng} markers")

    line_colors = sns.color_palette(line_palette, ng)

    fig, ax = plt.subplots()
    for i, grp in enumerate(groups):
        sub = df2[df2[z] == grp]
        xs, ys = sub[x].values, sub[y].values

        ax.plot(xs, ys,
                color=line_colors[i],
                linewidth=1.8,
                zorder=1)

        if is_num:
            ax.scatter(xs, ys,
                       c=sub[hue].values,
                       cmap=cmap, norm=norm,
                       marker=markers[i],
                       edgecolor='black',
                       s=50,
                       zorder=2)
        else:
            ax.scatter(xs, ys,
                       color=[lut[val] for val in sub[hue]],
                       marker=markers[i],
                       edgecolor='black',
                       s=50,
                       zorder=2)

        if annotate:
            for xi, yi in zip(xs, ys):
                ax.annotate(_wrapped_fmt(yi),
                            xy=(xi, yi),
                            xytext=(0, 5),
                            textcoords="offset points",
                            ha="center", va="bottom",
                            fontsize=DEFAULT_FONTSIZE_SMALL)

    if xlabel: ax.set_xlabel(xlabel, labelpad=8)
    if ylabel: ax.set_ylabel(ylabel, labelpad=8)

    xt = sorted(df2[x].unique())
    ax.set_xticks(xt)

    if ylim:
        ax.set_ylim(*ylim)
    else:
        y0, y1 = df2[y].min(), df2[y].max()
        pad = (y1 - y0)*0.1 if y1!=y0 else 1
        ax.set_ylim(y0-pad, y1+pad)

    ax.yaxis.set_major_formatter(FuncFormatter(lambda v,pos: _wrapped_fmt(v)))

    ztitle = z_label or z
    for i, grp in enumerate(groups):
        ax.plot([], [],
                color=line_colors[i],
                marker=markers[i],
                linestyle='-',
                linewidth=1.8,
                markersize=6,
                label=str(grp).upper())
    leg = ax.legend(title=ztitle, frameon=False, loc='lower right')
    title = leg.get_title()
    title.set_fontsize(DEFAULT_FONTSIZE_SMALL)

    if is_num:
        sm = ScalarMappable(norm=norm, cmap=cmap_obj)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label(hue_label or hue)

    sns.despine(top=True, right=True)
    fig.tight_layout()
    if save_path:
        fig.savefig(save_path)
    if show:
        plt.show()

## HQNN-Parallel

### HQNN-Parallel vs CNN

In [ ]:
df_p = read_csv_numeric_pandas(path="csv\\hqnn_parallel_results.csv")

Table: Model | Dataset | Test Accuracy [%] | (Params)

In [ ]:
df_p_table = filter_df(df_p, filters=
                       [
                            {
                                'model.name' : "hqnn-parallel",
                                'model.params.num_qlayers' : 4,
                                'model.params.num_qubits' : 5,
                                'model.params.num_qreps' : 4,
                                'Name' : lambda n : (n != "run_20250818_092157") & (n != "run_20250816_181237") # to ignore duplicates
                            },
                            {
                                'model.name' : "hqnn-parallel-classic-cnn"
                            }
                       ])

print_pretty_table(df=df_p_table,
                   columns=["Name", "model.name", "data.name", "test/accuracy"],
                   sort_by=["data.name", "model.name"])

Plot: loss(epoch) | accuracy(epoch)

In [ ]:
df_p_basic = read_csv_numeric_pandas(path="csv\\hqnn_parallel_basic_mnist_results.csv")

plot_2lines(
    df_p_basic,
    x="epoch",
    y1="train_loss",
    y2="val_loss",
    y1_label="Trening",
    y2_label="Walidacja",
    xlabel="Liczba epok",
    ylabel="Wartość funkcji kosztu",
    ylim=[0, 0.35],
    line_palette=["#3939dc" ,  "#ca3e3e"],
    legend_loc="upper right",
    show=True
)

plot_2lines(
    df_p_basic,
    x="epoch",
    y1="train_accuracy",
    y2="val_accuracy",
    y1_label="Trening",
    y2_label="Walidacja",
    y_multiplier=100,
    xlabel="Liczba epok",
    ylabel="Dokładność [%]",
    ylim=[0.94, 1.00075],
    line_palette=["#3939dc" ,  "#ca3e3e"],
    legend_loc="lower right",
    show=True
)

### HQNN-Parallel: qreps

Plot: accuracy(qreps)

In [ ]:
df_p_qreps = filter_df(df_p, filters=
                       [
                            {
                                'model.name' : "hqnn-parallel",
                                'model.params.num_qlayers' : 4,
                                'model.params.num_qubits' : 5,
                                'data.name' : "mnist",
                            }
                       ])

plot_bar(
    df_p_qreps,
    x="model.params.num_qreps",
    y="test/accuracy",
    hue="Params",
    y_multiplier=100.0,
    cmap="viridis",
    ylim=[0.9840, 0.9960],
    pallim=[45000, 45600],
    xlabel="Głębokość ansatzu",
    ylabel="Dokładność [%]",
    hue_label="Liczba parametrów",
    show=True
)

### HQNN-Parallel: qlayers

Plot: accuracy(qlayers)

In [ ]:
df_p_qlayers = filter_df(df_p, filters=
                         [
                            {
                                'model.name' : "hqnn-parallel",
                                'model.params.num_qreps' : 4,
                                'model.params.num_qubits' : 5,
                                'data.name' : "mnist",
                            }
                         ])

plot_bar(
    df_p_qlayers,
    x="model.params.num_qlayers",
    y="test/accuracy",
    hue="Params",
    y_multiplier=100.0,
    ylim=[0.9840, 0.9960],
    pallim=[20000, 90000],
    cmap="plasma",
    xlabel="Liczba warstw kwantowych",
    ylabel="Dokładność [%]",
    hue_label="Liczba parametrów",
    show=True
)

### HQNN-Parallel: qubits

Plot: accuracy(qubits)

In [ ]:
df_p_qubits = filter_df(df_p, filters=
                        [
                            {
                                'model.name' : "hqnn-parallel",
                                'model.params.num_qlayers' : 4,
                                'model.params.num_qreps' : 4,
                                'data.name' : "mnist",
                            }
                        ])

plot_bar(
    df_p_qubits,
    x="model.params.num_qubits",
    y="test/accuracy",
    hue="Params",
    y_multiplier=100.0,
    ylim=[0.9840, 0.9960],
    pallim=[20000, 80000],
    cmap="cividis",
    xlabel="Liczba kubitów",
    ylabel="Dokładność [%]",
    hue_label="Liczba parametrów",
    show=True
)

## HQNN-Quanv

### HQNN-Quanv vs CNN

In [ ]:
df_q = read_csv_numeric_pandas(path="csv\\hqnn_quanv_results.csv")

Table: Model | Dataset | Test Accuracy [%] | (Params)

In [ ]:
df_q_table = filter_df(df_q, filters=
                       [
                        {
                            'model.name' : "hqnn-quanv",
                            'model.params.ansatz_type' : "sel",
                            'model.params.num_qreps' : 3
                        },
                        {
                            'model.name' : "hqnn-quanv-classic-cnn"
                        }
                       ])

print_pretty_table(df=df_q_table,
                   columns=["Name", "model.name", "data.name", "test/accuracy"],
                   sort_by=["data.name", "model.name"])

Plot: loss(epoch) | accuracy(epoch)

In [ ]:
df_q_basic = read_csv_numeric_pandas(path="csv\\hqnn_quanv_basic_mnist_results.csv")

plot_2lines(
    df_q_basic,
    x="epoch",
    y1="train_loss",
    y2="val_loss",
    y1_label="Trening",
    y2_label="Walidacja",
    xlabel="Liczba epok",
    ylabel="Wartość funkcji kosztu",
    ylim=[0, 0.25],
    line_palette=["#3939dc" ,  "#ca3e3e"],
    legend_loc="upper right",
    show=True
)

plot_2lines(
    df_q_basic,
    x="epoch",
    y1="train_accuracy",
    y2="val_accuracy",
    y1_label="Trening",
    y2_label="Walidacja",
    y_multiplier=100,
    xlabel="Liczba epok",
    ylabel="Dokładność [%]",
    ylim=[0.93, 1.00075],
    line_palette=["#3939dc" ,  "#ca3e3e"],
    legend_loc="lower right",
    show=True
)

### HQNN-Quanv: ansatz and qreps

Plot x3: accuracy(ansatz,qreps)

In [ ]:
df_q_ans = filter_df(df_q, filters=
                        [
                            {
                                'model.name' : "hqnn-quanv",
                                'data.name' : "mnist",
                            }
                        ])

plot_3lines_hue(
    df_q_ans,
    x="model.params.num_qreps",
    y="test/accuracy",
    z="model.params.ansatz_type",
    hue="Params",
    y_multiplier=100.0,
    cmap="gist_yarg",
    ylim=[0.9300, 0.9800],
    pallim=[6780, 6900],
    annotate=False,
    line_palette=[ "#f6f63c", "#3939dc" ,  "#ca3e3e"],
    xlabel="Głębokość ansatzu",
    ylabel="Dokładność [%]",
    z_label="Ansatz",
    hue_label="Liczba parametrów",
    show=True
)


Plot: accuracy(qreps) for SEL

In [ ]:
df_q_sel = filter_df(df_q, filters=
                       [
                            {
                                'model.name' : "hqnn-quanv",
                                'model.params.ansatz_type' : "sel",
                                'data.name' : "mnist",
                            }
                       ])

plot_bar(
    df_q_sel,
    x="model.params.num_qreps",
    y="test/accuracy",
    hue="Params",
    y_multiplier=100.0,
    cmap="Reds",
    ylim=[0.9300, 0.9800],
    xlabel="Głębokość ansatzu",
    ylabel="Dokładność [%]",
    hue_label="Liczba parametrów",
    show=True
)

Plot: accuracy(qreps) for BEL

In [ ]:
df_q_bel = filter_df(df_q, filters=
                       [
                            {
                                'model.name' : "hqnn-quanv",
                                'model.params.ansatz_type' : "bel",
                                'data.name' : "mnist",
                            }
                       ])

plot_bar(
    df_q_bel,
    x="model.params.num_qreps",
    y="test/accuracy",
    hue="Params",
    y_multiplier=100.0,
    cmap="Blues",
    ylim=[0.9300, 0.9800],
    xlabel="Głębokość ansatzu",
    ylabel="Dokładność [%]",
    hue_label="Liczba parametrów",
    show=True
)

Plot: accuracy(qreps) for RL

In [ ]:
df_q_rl = filter_df(df_q, filters=
                       [
                            {
                                'model.name' : "hqnn-quanv",
                                'model.params.ansatz_type' : "rl",
                                'data.name' : "mnist",
                            }
                       ])

plot_bar(
    df_q_rl,
    x="model.params.num_qreps",
    y="test/accuracy",
    hue="Params",
    y_multiplier=100.0,
    cmap="YlOrBr",
    ylim=[0.9300, 0.9800],
    xlabel="Głębokość ansatzu",
    ylabel="Dokładność [%]",
    hue_label="Liczba parametrów",
    show=True
)